In [164]:
from selenium import webdriver
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.common.keys import Keys

from pandas import Series,DataFrame
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import random

from time import sleep
import re


# In[135]:



In [198]:
#로그인 부분
def login_seq(ids,passwords):
    try:
        browser.get("https://www.instagram.com/accounts/login/?source=auth_switcher")
        login = browser.find_element_by_name("username")
        login.send_keys(ids)

        password = browser.find_element_by_name("password")
        password.send_keys(passwords)

        submit = browser.find_elements_by_xpath("//*[@id='react-root']/section/main/div/article/div/div[1]/div/form/div[3]/button")

        AC(browser).move_to_element(submit[0]).click().perform()
    except:
        pass


# In[161]:


#검색어 입력
def search(keyword):

    browser.get("https://www.instagram.com/")
    search = browser.find_elements_by_xpath("//*[@id='react-root']/section/nav/div[2]/div/div/div[2]/input")
    sleep(1)
    search[0].send_keys(keyword)
    button = browser.find_elements_by_xpath("//*[@id='react-root']/section/nav/div[2]/div/div/div[2]/span")
    AC(browser).move_to_element(button[0]).click().perform()
    sleep(2)
    for i in browser.find_elements_by_tag_name("span"):
        if i.text == "#"+str(keyword):
            print("true")
            AC(browser).move_to_element(i).click().perform()
            break
        else : pass


# In[401]:


#블록 등 비정상상황 발생시 해당 게시물 위치부터 다시 시작

def fail_return(current):
    try:
        for ax in browser.find_elements_by_tag_name("a"):
            if ax.find_element_by_xpath("..").find_element_by_xpath("..").find_element_by_xpath("..").find_element_by_xpath("..").find_element_by_xpath("..").tag_name == "article":
                tests = ax.get_attribute("href").split("/")[4]
                if current == tests:
                    AC(browser).move_to_element(ax).click().perform()
    except: pass



# In[500]:


#엔진 속 엔진. 실제 데이터들을 스크래핑하여 datus 변수로 이를 반환(return)함

def engine_engine(current):
    datus = list()
    log = list()
    fail_return(current)
    log.append("소생 페이즈 성공")
    
    for i in range(0,23):
        if i <= 12:
            log.append("start,"+str(i))
            j = len(browser.find_elements_by_xpath("/html/body/div[3]/div/div[1]/div/div/a"))
            for k in browser.find_elements_by_xpath("/html/body/div[3]/div/div[1]/div/div/a"):

                #만일 이전, 다음 버튼 두개가 존재하는 경우
                if j == 2:
                    if k.text == "다음" :
                        sleep(1)
                        ab = browser.find_element_by_xpath("/html/body/div[3]/div/div[2]/div/article/div[2]/div[1]/ul/li[1]/div/div/div/span")
                        datus.append(ab.text.replace("\n","").replace("#"," "))
                        AC(browser).move_to_element(k).click().perform()
                        current = browser.current_url.split("/")[4]
                    
                #만일 이전, 다음 버튼 중 하나만 존재하는 경우
                if j == 1:
                    #다음 버튼만 존재하는 경우 : 첫 페이지를 의미
                    if k.text == "다음":
                        sleep(1)
                        ab = browser.find_element_by_xpath("/html/body/div[3]/div/div[2]/div/article/div[2]/div[1]/ul/li[1]/div/div/div/span")
                        datus.append(ab.text.replace("\n","").replace("#"," "))
                        AC(browser).move_to_element(k).click().perform()
                        current = browser.current_url.split("/")[4]
                    
                    #이전 버튼만 존재하는 경우 : 마지막 페이지를 의미
                    else :
                        sleep(1)
                        ab = browser.find_element_by_xpath("/html/body/div[3]/div/div[2]/div/article/div[2]/div[1]/ul/li[1]/div/div/div/span")
                        datus.append(ab.text.replace("\n","").replace("#"," "))
                        exit = browser.find_element_by_xpath("/html/body/div[3]/div/button")
                        AC(browser).move_to_element(exit).click().perform()
                        current = browser.current_url.split("/")[4]
        if i >= 12: 
            log.append("end,"+str(i))
            log.append("종료 페이즈 시작")
            exit = browser.find_element_by_xpath("/html/body/div[3]/div/button")
            AC(browser).move_to_element(exit).click().perform()
            AC(browser).send_keys(Keys.PAGE_DOWN).perform()
            return (datus,current,log)


# In[535]:


#엔진. 정상 진행과 예외 진행을 구분하여 예외 발생 시 이를 핸들링해준다.

def engine(firstpage,lastpage,current):
    datalist = []
    kill_switch = 0
    log = list()
    
    while firstpage <= lastpage:
        sleep(random.randint(0,1))
        #kill_switch = 0, 즉 예외가 발생했던 적이 없는 경우
        if kill_switch == 0:
            try : 
                aa,current,log = engine_engine(current)
                datalist.append(aa)
                log += log
                firstpage += 1
            #예외가 발생하면 일단 창의 X버튼을 누르고, kill_switch를 1로 켠다. 
            except :
                try:
                    exit = browser.find_element_by_xpath("/html/body/div[3]/div/button")
                    AC(browser).move_to_element(exit).click().perform()
                except : pass
                kill_switch = 1
                log.append("킬 스위치 켜짐")
                sleep(1)
        #kill_switch = 1, 즉 예외가 발생했던 경우
        if kill_switch == 1:
            #fail_return으로 마지막 위치에서 다시 시작한다.
            try:
                aa,current,log = engine_engine(current)
                datalist.append(aa)
                log += log

                #예외 처리 후 kill_switch를 0으로 다시 끈다.
                kill_switch = 0
                log.append("킬 스위치 꺼짐")
                firstpage += 1
            except:
                exit = browser.find_element_by_xpath("/html/body/div[3]/div/button")
                AC(browser).move_to_element(exit).click().perform()
                sleep(1)
    return (datalist,current,log)


# In[498]:


#껍데기(미사용)

def main(ids,passwords,keyword,lastpage):
    datum = list()
    
    login_seq(ids,passwords)
    sleep(1)
    
    search(keyword)
    sleep(7)
    
    cont = browser.find_element_by_xpath("//*[@id='react-root']/section/main/article/div[2]/div/div[1]/div[1]/a")
    current = cont.get_attribute("href").split("/")[4]
    
    datum.append(engine(lastpage,current))
    return datum
    




In [199]:
browser = webdriver.Chrome("chromedriver.exe")

In [200]:
# In[541]:


datum = list()
logfile = DataFrame()

login_seq("neverland251@gmail.com","cjswp12358")
sleep(1)

search("하이네켄")
sleep(3)

cont = browser.find_element_by_xpath("//*[@id='react-root']/section/main/article/div[2]/div/div[1]/div[1]/a")
current = cont.get_attribute("href").split("/")[4]
fail_return(current)

for i in range(0,7600):
    aaa,current,log = engine(1,3,current)    
    datum.append(aaa)
    log += log
    logfile = pd.concat([logfile,DataFrame(log)])

logfile.reset_index()
logfile.to_csv("log.csv")

IndexError: list index out of range

In [194]:
datum[8][0][11]

' 두번째술집기가막힌 막걸리를 먹고싶다면?우리 9시에 만나요!!늦지마요~—————————————————- 5주년 &  송년회안녕하세요.성인들의 건강한 운동놀이터 K 헤드코치 김동석 입니다.수없이 많은 좋은 분들과 소통하고 지낼수 있었던 5년 이었습니다. 너무너무 행복했습니다.진심으로 고개숙여 감사인사 드립니다.저희 케이는 앞으로도 운동이라는 매개체로 여러분들에게 한걸음 더 다가갈수 있도록 노력하겠습니다.바쁘신 와중에도 가벼운 발걸음 해주셨음 합니다. 두손은 무겁게ㅋ오늘 늦은 9시부터 행사가 시작될 예정이오니 늦어도 10분전에는 입장 부탁드립니다.감사합니다.동코, 여코, 장코, 준코 올림 두번째술집  퍼니비어  청춘횟집  쇼간지  별박스  와와돈까스  무브먼트  락테이프  와드프렌즈  하이네켄  칠번가피자——————————————————🔥 8주 서바이벌 지독한 다이어트 18기 종료 🏋️\u200d♂️🔥 코치진 @bityeoul @jey_hong @charles.y.jun🔥 남녀노소 누구나 쉽게 할수 있는 운동 🏋🏻\u200d♂️🔥 운동은 기본! 웃음치료도 가능해요👍🔥 시작이 반이다. 우선 시작해 보세요! 저희가 있잖아요^^ ——————————————————🔥 1일 무료 체험 / 문의 > DM / 카톡🔥 상담문의 : 070-7694-0012🔥 네이버카페 : 크로스핏케이🔥 위치 : 경성대/부경대역 5번 출구 1분거리——————————————————🔥 학생/경찰/군인/소방 상시할인🔥 공무원체력,체대준비생,속성다이어트🔥 선수트레이닝, 근골격계 질환 개선'

In [307]:
data = []
datat = list()

In [339]:
def datatest():
    data = list()
    data.append("aa")
    return data

In [352]:
def datatest2(data):
    datum = list()
    datum.append(data)
    return datum

In [361]:
datalist = datatest2(data)

In [363]:
datalist[0]

['aa']

In [94]:
data = pd.read_csv("test.csv")

In [163]:
len(data.loc[2][3].split(","))

13

In [365]:
datum

[]